1. 이미지분석
2. custom vision- clssification
3. face 
4. OCR
5. Video Indexer

# Azure AI Vision 솔루션

* .NET 패키지 설치

In [2]:
#r "nuget: Microsoft.Extensions.Configuration.Json, 3.1.3" 
#r "nuget: System.Drawing.Common, 4.7.0" 
#r "nuget: Microsoft.Azure.CognitiveServices.Vision.Face, 2.8.0-preview.3"
#r "nuget: Azure.AI.Vision.ImageAnalysis, 1.0.0-beta.2" 

Installed Packages Azure.AI.Vision.ImageAnalysis, 1.0.0-beta.2 Microsoft.Azure.CognitiveServices.Vision.Face, 2.8.0-preview.3 Microsoft.Extensions.Configuration.Json, 3.1.3 System.Drawing.Common, 4.7.0

* namespace 가져오기

In [23]:
using System;
using System.IO;
using System.Net.Http;
using System.Net.Http.Headers;
using System.Text;
using System.Text.Json;
using System.Threading.Tasks;
using System.Drawing;
using Microsoft.Extensions.Configuration;
using Azure;
using System.Linq;
using System.Collections.Generic;
using System.Threading;
using Azure.AI.Vision.ImageAnalysis;
using Microsoft.Azure.CognitiveServices.Vision.Face;
using Microsoft.Azure.CognitiveServices.Vision.Face.Models;

# 1. 이미지 분석

In [4]:
// 사용할 이미지의 주소 가져오기
string address_1="C:\\Users\\parkm\\Desktop\\Vision\\mslearn-ai-vision\\Labfiles\\01-analyze-images\\C-Sharp\\image-analysis\\";
public string imageFile = address_1+"images/street.jpg";

In [7]:
//API 사용을 위한 인증 절차
public string aiSvcEndpoint = "https://fruit-train.cognitiveservices.azure.com/";
public string aiSvcKey ="68309624561e49688c4cbc6143304744";

public var client = new ImageAnalysisClient(new Uri(aiSvcEndpoint), new AzureKeyCredential(aiSvcKey));

In [8]:
//이미지 분석 API 처리 함수
public static void AnalyzeImage(string imageFile, ImageAnalysisClient client)
        {
            Console.WriteLine($"\nAnalyzing {imageFile} \n");

            // 이미지 파일을 전달할 수 있도록 파일 스트림 형태로 바꾸기
            using FileStream stream = new FileStream(imageFile,
                                                     FileMode.Open);

            // 이미지 분석 결과 가져오기
            ImageAnalysisResult result = client.Analyze(
                BinaryData.FromStream(stream),
                VisualFeatures.Caption | VisualFeatures.DenseCaptions | VisualFeatures.Objects | VisualFeatures.Tags | VisualFeatures.People);
                        
            
            // 가장 정확한 이미지 분석 결과 출력
            if (result.Caption.Text != null)
            {
                Console.WriteLine(" Caption:");
                Console.WriteLine($"   \"{result.Caption.Text}\", 신뢰도 {result.Caption.Confidence:0.00}\n");
            }

            // 이미지에 대한 다양한 분석 결과 출력
            Console.WriteLine(" Dense Captions:");
            foreach (DenseCaption denseCaption in result.DenseCaptions.Values)
            {
                Console.WriteLine($"   Caption: '{denseCaption.Text}', 신뢰도: {denseCaption.Confidence:0.00}");
            }

        }

In [9]:
// 이미지 분석
AnalyzeImage(imageFile, client);


Analyzing C:\Users\parkm\Desktop\Vision\mslearn-ai-vision\Labfiles\01-analyze-images\C-Sharp\image-analysis\images/street.jpg 

 Caption:
   "a man walking a dog on a leash on a street", 신뢰도 0.82

 Dense Captions:
   Caption: 'a man walking a dog on a leash on a street', 신뢰도: 0.82
   Caption: 'a man walking on a street', 신뢰도: 0.69
   Caption: 'a yellow car on the street', 신뢰도: 0.78
   Caption: 'a black dog walking on the street', 신뢰도: 0.75
   Caption: 'a blurry image of a blue car', 신뢰도: 0.82
   Caption: 'a yellow taxi cab on the street', 신뢰도: 0.72


In [10]:
 // 이미지 배경화면 제거 API 처리 함수
public static async Task BackgroundForeground(string imageFile, string endpoint, string key){
           
            Console.WriteLine($" Background removal:");
            // API 버전과 모드 지정
            string apiVersion = "2023-02-01-preview";
            string mode = "backgroundRemoval"; // "foregroundMatting" 또는 "backgroundRemoval" 지정 가능
            string url = $"computervision/imageanalysis:segment?api-version={apiVersion}&mode={mode}";

            // API 호출
            using (var client = new HttpClient())
            {
                var contentType = new MediaTypeWithQualityHeaderValue("application/json");
                client.BaseAddress = new Uri(endpoint);
                client.DefaultRequestHeaders.Accept.Add(contentType);
                client.DefaultRequestHeaders.Add("Ocp-Apim-Subscription-Key", key);

                var data = new{
                    url = $"https://github.com/MicrosoftLearning/mslearn-ai-vision/blob/main/Labfiles/01-analyze-images/Python/image-analysis/images/street.jpg?raw=true"
                };

                var jsonData = JsonSerializer.Serialize(data);
                var contentData = new StringContent(jsonData, Encoding.UTF8, contentType);
                var response = await client.PostAsync(url, contentData);

                File.WriteAllBytes("background.png", response.Content.ReadAsByteArrayAsync().Result);
                Console.WriteLine("  Results saved in background.png\n");
                
            }
            
}

In [11]:
// 배경 제거
await BackgroundForeground(imageFile, aiSvcEndpoint, aiSvcKey);

 Background removal:
  Results saved in background.png



# 3.얼굴 감지

* Computer Vision (얼굴 이정표)

In [19]:
// 사용할 이미지의 주소 가져오기
public string imageFile = "C:/Users/parkm/Desktop/Vision/mslearn-ai-vision/Labfiles/04-face/C-Sharp/computer-vision/images/people.jpg";

In [13]:
//API 사용을 위한 인증 절차
var aiSvcEndpoint = "https://fruit-train.cognitiveservices.azure.com/";
var aiSvcKey = "3446cc4f896148e79b4864c0d86fd497";

public var client = new ImageAnalysisClient(
    new Uri(aiSvcEndpoint),
    new AzureKeyCredential(aiSvcKey)
);

In [14]:
//얼굴 이정표 분석 API 처리 함수
public static void AnalyzeImage(string imageFile, ImageAnalysisClient client)
{
    Console.WriteLine($"\nAnalyzing {imageFile} \n");

    // 이미지 파일을 전달할 수 있도록 파일 스트림 형태로 바꾸기
    using FileStream stream = new FileStream(imageFile, FileMode.Open);

    // 이미지에 있는 사람 감지 결과 가져오기
    ImageAnalysisResult result = client.Analyze(
        BinaryData.FromStream(stream),
        VisualFeatures.People);

    // 감지 결과 출력하기
    Console.WriteLine($"이미지에서의 얼굴 분석 결과:");
    Console.WriteLine($" Metadata: Model: {result.ModelVersion} Image dimensions: {result.Metadata.Width} x {result.Metadata.Height}");
    Console.WriteLine($" 분석된 사람들:");
    foreach (DetectedPerson person in result.People.Values)
    {
        if(person.Confidence > 0.3)
            Console.WriteLine($"   사람 : Bounding box {person.BoundingBox.ToString()}, 정확도 {person.Confidence:F4}");
    }
}

In [15]:
//얼굴 이정표 분석
AnalyzeImage(imageFile, client);


Analyzing C:/Users/parkm/Desktop/Vision/mslearn-ai-vision/Labfiles/04-face/C-Sharp/computer-vision/images/people.jpg 

이미지에서의 얼굴 분석 결과:
 Metadata: Model: 2023-10-01 Image dimensions: 800 x 514
 분석된 사람들:
   사람 : Bounding box Top: 73, Left: 454, Width: 345, Height: 439, 정확도 0.9518
   사람 : Bounding box Top: 0, Left: 87, Width: 491, Height: 513, 정확도 0.9195


* Face API (얼굴 특징 분석)

In [20]:
// 사용할 이미지의 주소 가져오기
public string imageFile = "C:/Users/parkm/Desktop/Vision/mslearn-ai-vision/Labfiles/04-face/C-Sharp/face-api/images/people.jpg";

In [24]:
//API 사용을 위한 인증 절차
string cogSvcEndpoint = "https://fruit-train.cognitiveservices.azure.com/";
string cogSvcKey = "3446cc4f896148e79b4864c0d86fd497";

Microsoft.Azure.CognitiveServices.Vision.Face.ApiKeyServiceClientCredentials credentials = new Microsoft.Azure.CognitiveServices.Vision.Face.ApiKeyServiceClientCredentials(cogSvcKey);

public FaceClient client = new FaceClient(credentials)
{
    Endpoint = cogSvcEndpoint
};

In [25]:
//얼굴 감지 결과 출력 함수
public static void DrawFaceAnnotations(Image image, IList<DetectedFace> detected_faces){
        Graphics graphics = Graphics.FromImage(image);
        Pen pen = new Pen(Color.LightGreen, 3);
        Font font = new Font("Arial", 14);
        SolidBrush brush = new SolidBrush(Color.White);
        int faceCount = 0;

        foreach (var face in detected_faces){
            faceCount++;
            Console.WriteLine($"\nFace number {faceCount}");

            Console.WriteLine($" - 입을 다물고 있는지 여부: {face.FaceAttributes.Occlusion.MouthOccluded}");
            Console.WriteLine($" - 눈을 감고 있는지 여부: {face.FaceAttributes.Occlusion.EyeOccluded}");
            Console.WriteLine($" - 블러 정도: {face.FaceAttributes.Blur.BlurLevel}");
            Console.WriteLine($" - 안경 착용 여부: {face.FaceAttributes.Glasses}");

            var r = face.FaceRectangle;
            Rectangle rect = new Rectangle(r.Left, r.Top, r.Width, r.Height);
            graphics.DrawRectangle(pen, rect);
            string annotation = $"Face number {faceCount}";
            graphics.DrawString(annotation, font, brush, r.Left, r.Top);
        }
    }


In [26]:
//분석 결과 이미지로 저장하는 함수
public static void SaveAnnotatedImage(Image image, string output_file){
        image.Save(output_file);
        Console.WriteLine("Results saved in " + output_file);
}

In [27]:
//얼굴 분석 API 호출 함수
public static async Task DetectFaces(string imageFile, FaceClient client){
        Console.WriteLine($"Detecting faces in {imageFile}");

        // API에게 요청할 내용 설정
        IList<FaceAttributeType> features = new FaceAttributeType[]{
            FaceAttributeType.Occlusion, //얼굴 부분 가리는 정도
            FaceAttributeType.Blur, //흐림 정도
            FaceAttributeType.Glasses //안경 착용 여부
        };

        using (var imageData = File.OpenRead(imageFile)){
            var detected_faces = await client.Face.DetectWithStreamAsync(imageData, returnFaceAttributes: features, returnFaceId: false);

            if (detected_faces.Count() > 0){
                Console.WriteLine($"{detected_faces.Count()} faces detected.");

                Image image = Image.FromFile(imageFile);
                DrawFaceAnnotations(image, detected_faces);

                string output_file = "detected_faces.jpg";
                SaveAnnotatedImage(image, output_file);
            }
        }
}


In [28]:
//얼굴 특징 분석 
await DetectFaces(imageFile, client);

Detecting faces in C:/Users/parkm/Desktop/Vision/mslearn-ai-vision/Labfiles/04-face/C-Sharp/face-api/images/people.jpg
2 faces detected.

Face number 1
 - 입을 다물고 있는지 여부: False
 - 눈을 감고 있는지 여부: False
 - 블러 정도: Low
 - 안경 착용 여부: NoGlasses

Face number 2
 - 입을 다물고 있는지 여부: False
 - 눈을 감고 있는지 여부: False
 - 블러 정도: Low
 - 안경 착용 여부: ReadingGlasses
Results saved in detected_faces.jpg


# 4. OCR

In [31]:
//API 사용을 위한 인증 절차
public string aiSvcEndpoint = "https://fruit-train.cognitiveservices.azure.com/";
public string aiSvcKey = "68309624561e49688c4cbc6143304744";


public ImageAnalysisClient client = new ImageAnalysisClient(
    new Uri(aiSvcEndpoint),
    new AzureKeyCredential(aiSvcKey)
);

In [37]:
// 분석 결과 출력 함수
public static void DrawTextAndSaveImage(ImageAnalysisResult result, System.Drawing.Image image, Graphics graphics, Pen pen){
    foreach (var line in result.Read.Blocks.SelectMany(block => block.Lines)){
        // 분석 결과 텍스트로 출력
        Console.WriteLine($"   '{line.Text}'");
        var drawLinePolygon = true;
        Console.WriteLine($"   바운딩 영역 : [{string.Join(" ", line.BoundingPolygon)}]");   
        foreach (DetectedTextWord word in line.Words){
            Console.WriteLine($"     단어: '{word.Text}', 신뢰도 {word.Confidence:F4}, 바운딩 영역: [{string.Join(" ", word.BoundingPolygon)}]");             
            drawLinePolygon = false;
            var r = word.BoundingPolygon;
            Point[] polygonPoints = {
                new Point(r[0].X, r[0].Y), new Point(r[1].X, r[1].Y), new Point(r[2].X, r[2].Y), new Point(r[3].X, r[3].Y)
            };
            graphics.DrawPolygon(pen, polygonPoints);
        }
        // 이미지 내에서 텍스트가 감지된 영역 정보 저장
        if (drawLinePolygon){
            var r = line.BoundingPolygon;
            Point[] polygonPoints = {
                new Point(r[0].X, r[0].Y), new Point(r[1].X, r[1].Y), new Point(r[2].X, r[2].Y), new Point(r[3].X, r[3].Y)
            };
            graphics.DrawPolygon(pen, polygonPoints);
        }
    }
}

In [38]:
//이미지 위에 텍스트 영역 표시 및 저장 함수
public static void DisplayAndSaveResults(string imageFile, ImageAnalysisResult result){
    Console.WriteLine($"Text:");
    System.Drawing.Image image = System.Drawing.Image.FromFile(imageFile);
    Graphics graphics = Graphics.FromImage(image);
    Pen pen = new Pen(Color.Cyan, 3);
    
    DrawTextAndSaveImage(result, image, graphics, pen);

    // 분석 내용이 반영된 이미지 저장
    string output_file = "text.jpg";
    image.Save(output_file);
    Console.WriteLine("\nResults saved in " + output_file + "\n");
}

In [39]:
//OCR API 호출 함수
public static void GetTextRead(string imageFile, ImageAnalysisClient client){
    Console.WriteLine($"\nReading text from {imageFile} \n");

    // 이미지를 파일 스트림 형태로 변환
    using FileStream stream = new FileStream(imageFile, FileMode.Open);

    // 이미지 분석 결과 받기
    ImageAnalysisResult result = client.Analyze(
        BinaryData.FromStream(stream),
        VisualFeatures.Read);
                
    stream.Close();
                
    // 분석 결과 보여주기
    DisplayAndSaveResults(imageFile, result);
}

In [35]:
// 사용할 이미지의 주소 일부 가져오기
 public static String address_3="C:\\Users\\parkm\\Desktop\\Vision\\mslearn-ai-vision\\Labfiles\\05-ocr\\C-Sharp\\read-text\\images\\";

In [36]:
//1. Lincoln.jpg 읽기
GetTextRead(address_3+"Lincoln.jpg", client);


Reading text from C:\Users\parkm\Desktop\Vision\mslearn-ai-vision\Labfiles\05-ocr\C-Sharp\read-text\images\Lincoln.jpg 

Text:
   'IN THIS TEMPLE'
   Bounding Polygon: [(328, 171) (477, 169) (477, 184) (328, 186)]
     Word: 'IN', Confidence 0.9930, Bounding Polygon: [(328, 171) (342, 171) (342, 187) (328, 187)]
     Word: 'THIS', Confidence 0.9910, Bounding Polygon: [(357, 171) (397, 170) (397, 185) (357, 186)]
     Word: 'TEMPLE', Confidence 0.9930, Bounding Polygon: [(407, 170) (474, 170) (474, 184) (407, 185)]
   'AS IN THE HEARTS OF THE PEOPLE'
   Bounding Polygon: [(240, 193) (564, 188) (564, 203) (240, 210)]
     Word: 'AS', Confidence 0.9950, Bounding Polygon: [(241, 194) (262, 194) (262, 210) (241, 211)]
     Word: 'IN', Confidence 0.9980, Bounding Polygon: [(270, 193) (284, 193) (284, 210) (270, 210)]
     Word: 'THE', Confidence 0.9930, Bounding Polygon: [(298, 193) (332, 192) (332, 208) (298, 209)]
     Word: 'HEARTS', Confidence 0.9940, Bounding Polygon: [(340, 192) (410,

In [40]:
//2.Note.jpg 읽기
GetTextRead(address_3+"Note.jpg", client);


Reading text from C:\Users\parkm\Desktop\Vision\mslearn-ai-vision\Labfiles\05-ocr\C-Sharp\read-text\images\Note.jpg 

Text:
   'Shopping List'
   바운딩 영역 : [(231, 141) (693, 147) (691, 245) (230, 240)]
     단어: 'Shopping', 신뢰도 0.9630, 바운딩 영역: [(240, 141) (535, 149) (531, 245) (234, 234)]
     단어: 'List', 신뢰도 0.8300, 바운딩 영역: [(554, 149) (689, 147) (686, 244) (550, 245)]
   'Non- Fat milk'
   바운딩 영역 : [(149, 302) (633, 297) (633, 374) (150, 378)]
     단어: 'Non-', 신뢰도 0.5770, 바운딩 영역: [(150, 303) (309, 301) (310, 378) (153, 378)]
     단어: 'Fat', 신뢰도 0.8420, 바운딩 영역: [(324, 301) (438, 300) (437, 378) (325, 378)]
     단어: 'milk', 신뢰도 0.9940, 바운딩 영역: [(476, 299) (620, 298) (617, 374) (475, 377)]
   'Bread'
   바운딩 영역 : [(138, 400) (382, 399) (383, 472) (138, 474)]
     단어: 'Bread', 신뢰도 0.9950, 바운딩 영역: [(152, 400) (366, 400) (368, 471) (151, 475)]
   'Eggs'
   바운딩 영역 : [(146, 517) (351, 526) (348, 605) (146, 609)]
     단어: 'Eggs', 신뢰도 0.9920, 바운딩 영역: [(149, 517) (342, 519) (341, 610) (148, 609)]